# Classifier SHAP Utilities

## Import Dependencies

In [1]:
import os

import numpy as np
import pandas as pd
import shap
import torch
from IPython.core.display import display, HTML
from adept.preprocess import CPUPreprocessor, GPUPreprocessor
from adept.preprocess.ops import FromNumpy, CastToFloat
from adept.utils.util import DotDict
from pysc2.lib.features import FeatureUnit
from pysc2.lib.upgrades import Upgrades

from gamebreaker.classifier.utils.common_utils import (
    create_upgrades_vec, 
    get_path_from_args,
    get_old_args,
    get_network
)
from gamebreaker.env.base.ops import (
    CreateEmptyArray,
    ProcessCategoricalFeatures,
    CopyOverScalarFeatures,
    FilterForNet,
    ProcessScalarFeatures,
    PermuteToFS,
)
from gamebreaker.env.base.obs_idx import ObsIdx, CustomObsIdx
display(HTML("<style>.container {width:100% !important;}</style>"))
shap.initjs()

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


## Build the Model

In [3]:
args = {
    "logdir": "/media/banshee/gb_winprob/refactored_models",
    "tag": "oops_all_noise",
    "gpu_id": 0,
}

args = DotDict(args)

# Grab the model path from the arguments (if it says it created one, there's something wrong with your
# args)
model_path = get_path_from_args(args)

# Grab the old arguments and merge in our new ones
args = get_old_args(args, model_path)

# Build the network
network = get_network(args, model_path)
network.eval()

ModularNetwork(
  (source_nets): ModuleDict(
    (units): WinProbConv(
      (in_conv): Conv1d(130, 32, kernel_size=(1,), stride=(1,))
      (convs): ModuleList(
        (0): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
        (1): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
        (2): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
        (3): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
        (4): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
        (5): Conv1d(32, 32, kernel_size=(3,), stride=(2,), padding=(1,))
      )
      (in_norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (norms): ModuleList(
        (0): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_s

## Create a Wrapper Around Forward()

### Build the Preprocessors

In [10]:
# X and Y max for the map size
x_max = 64
y_max = 64

observation_space = {"raw_units": (None, len(FeatureUnit))}
observation_dtypes = {"raw_units": np.int64}

unit_max = 512

cpu_preprocessor = CPUPreprocessor(
    [
        FromNumpy("raw_units", "raw_units"),
        CreateEmptyArray(unit_max, "raw_units", "proc_units", creep=True),
        ProcessCategoricalFeatures(
            x_max, y_max, ["raw_units", "proc_units"], ["proc_units"]
        ),
        CopyOverScalarFeatures(
            ["raw_units", "proc_units"], ["proc_units"], creep=True
        ),
        FilterForNet("raw_units", "raw_units"),
    ],
    observation_space,
    observation_dtypes,
)
gpu_preprocessor = GPUPreprocessor(
    [
        ProcessScalarFeatures(
            x_max, y_max, "proc_units", "proc_units", creep=True
        ),
        CastToFloat("proc_units", "proc_units"),
        PermuteToFS("proc_units", "proc_units"),
    ],
    cpu_preprocessor.observation_space,
    cpu_preprocessor.observation_dtypes,
)


### Define the wrapper for the network's `forward()`function

In [17]:
# The index in the SHAP vector where the upgrades list starts
upgrade_idx = 2 * len(Upgrades)

UNIT_MAX = 512
NB_FEATURES = len(FeatureUnit)
key = "win"


# This function will undo the SHAP transformations, preprocess the data
# and then send it through the network
@torch.no_grad()
def SHAP_forward(input_data: np.ndarray):
    # Grab all of the unit information from the 1-dimensional input_data
    unit_data = input_data[:, 0 : UNIT_MAX * NB_FEATURES]

    # Reshape the unit data to the shape expected by the network
    unit_data = np.reshape(unit_data, (-1, UNIT_MAX, NB_FEATURES))

    # Grab all of the upgrade information from the 1-dimensional input_data
    upgrade_data = input_data[:, UNIT_MAX * NB_FEATURES:]

    # CPU Preprocessor doesn't support batched data, so instead we predict in serial
    # TODO: fix this
    predictions = torch.zeros(unit_data.shape[0])
    gpu_obs = {"raw_units": None}
    for idx, timestep in enumerate(unit_data):
        obs = {"raw_units": timestep}
        obs = cpu_preprocessor(obs)
        obs = {
            "proc_units": obs["proc_units"].reshape(
                (1, *obs["proc_units"].shape)
            )
        }

        obs = gpu_preprocessor(obs)

        upgrade_list = torch.from_numpy(upgrade_data[idx])
        upgrade_list = torch.reshape(upgrade_list, (-1, upgrade_list.shape[0]))
        upgrade_list = upgrade_list.float().to(args.gpu_id)
        
        prediction = {}
        
        temp = network.forward(
            {"units": obs["proc_units"].float().to(args.gpu_id), "upgrades": upgrade_list},
            {}
        )

        # SHAP doesn't handle output vectors super well (considers them all classification outputs), so we can only
        # deal with one output at a time
        pred = temp[0][key].squeeze(-1)
        pred = torch.sigmoid(pred)
        pred = pred.double()
        predictions[idx] = pred

    # SHAP expects CPU numpy values
    return predictions.cpu().numpy()


In [12]:
# Next, SHAP typically implies a need for 1D data, so we create a 
# function that will transform our raw data into SHAP-friendly data
def get_shap_input(unit_data: np.ndarray, upgrade_data: np.ndarray) -> np.ndarray:
    flattened_unit_data = unit_data.flatten()
    
    shap_input_array  = np.zeros((flattened_unit_data.shape[-1] + upgrade_data.shape[-1]))
    
    shap_input_array[0:flattened_unit_data.shape[-1]] = flattened_unit_data
    shap_input_array[flattened_unit_data.shape[-1]:] = upgrade_data
    
    return shap_input_array

In [ ]:
import os
import numpy as np

from gamebreaker.classifier.utils.common_utils import add_one_hot_vector

from pysc2.lib.features import FeatureUnit

raw_path = "/media/banshee/gb_winprob/Data/raw_dataset_v3/Testing/0"
raw_files = [os.path.join(raw_path, file) for file in os.listdir(raw_path)]
raw_files.sort()

for file in raw_files:
    print(file)
    game_data = np.load(file, allow_pickles=True)[0]
    

## Load the Data

In [3]:
# Load the data (presaved into SHAP format) from Banshee
shap_data = np.load("/media/banshee/gb_winprob/Data/SHAP_dataset_v3_no_terrain/unit_data.npy", allow_pickle=True)

# # Just run a quick run of predictions to show that this function works
# with torch.no_grad():
#     predictions = SHAP_forward(shap_data[0:10])
#     print(predictions)

In [19]:
import os
from gamebreaker.agent import AGENT_LIST
from gamebreaker.classifier.utils.common_utils import add_one_hot_vector
from pysc2.lib import units
from pysc2.lib.features import PlayerRelative

raw_datapath = "/media/banshee/gb_winprob/Data/raw_dataset_v3/Testing/0"
raw_files = os.listdir(raw_datapath)
raw_files = sorted(raw_files)

for file in raw_files:
    # load the data in
    game_data = np.load(os.path.join(raw_datapath, file), allow_pickle=True)[0]
    upgrade_data = game_data["upgrades"]
    unit_data = game_data["data"]
    agent_data = game_data["agents"]

    # Determine player races
    blue_units = [
        unit
        for unit in unit_data[0]
        if unit[FeatureUnit.alliance] == PlayerRelative.SELF
    ]
    
    red_units = [
        unit
        for unit in unit_data[0]
        if unit[FeatureUnit.alliance] == PlayerRelative.ENEMY
    ]
    
    if blue_units[0][FeatureUnit.unit_type] in list(units.Terran):
        blue_race = 0
    elif blue_units[0][FeatureUnit.unit_type] in list(units.Protoss):
        blue_race = 1
    else:
        blue_race = 2
        
    if red_units[0][FeatureUnit.unit_type] in list(units.Terran):
        red_race = 0
    elif red_units[0][FeatureUnit.unit_type] in list(units.Protoss):
        red_race = 1
    else:
        red_race = 2
        
    # Generate the context vector
    upgrade_vec = torch.from_numpy(create_upgrades_vec(upgrade_data))
    upgrade_vec = torch.reshape(upgrade_vec, (1, -1))
    
    races_vec = add_one_hot_vector(
        upgrade_vec, torch.tensor([[blue_race, red_race]]), 3
    )
    
    blue_agent_ix = [
        ix
        for ix, agent_name in enumerate(AGENT_LIST)
        if agent_data[0] in agent_name
    ]
    
    red_agent_ix = [
        ix
        for ix, agent_name in enumerate(AGENT_LIST)
        if agent_data[1] in agent_name
    ]
    
    context_vec = add_one_hot_vector(
        races_vec, torch.tensor([[blue_agent_ix[0], red_agent_ix[0]]]), len(AGENT_LIST)
    ).numpy()
    break

In [21]:
context_vec.shape

(1, 192)

## Create the Kernel Explainer

In [ ]:
with torch.no_grad():
    explainer = shap.KernelExplainer(
        SHAP_forward,
        shap_data[0:10]
    )

## Load in the Test Data

In [ ]:
# Path to Raw PySC2 games
test_path = "/media/banshee/gb_winprob/Data/raw_dataset_v3_no_terrain/Validation/0/"
test_files = sorted(os.listdir(test_path))

shap_samples = None
for file in test_files[0:10]:
    # Load the data
    test_data = np.load(os.path.join(test_path, file), allow_pickle=True)[0]
    unit_data = np.zeros((512, 47))
    
    # Move the unit data of the first timestep into a np.ndarray with normalized unit dimension
    unit_data[0:test_data["data"][0].shape[0]:] = test_data["data"][0]
    
    # Create a vector for the upgrade information
    upgrade_data = create_upgrades_vec(
        [
            test_data["upgrades"][0],
            test_data["upgrades"][1]
        ]
    )
    
    # Append to an np.ndarray
    if shap_samples is not None:
        shap_samples = np.append(
            shap_samples, 
            np.reshape(get_shap_input(unit_data, upgrade_data), (1, -1)), 
            axis=0
        )
    else:
        shap_samples = np.reshape(get_shap_input(unit_data, upgrade_data), (1, -1))
        

# Create the labels for the pandas dataframe
unit_labels = []
for unit_number in range(512):
    for feature in FeatureUnit:
        unit_labels.append(f"Unit {unit_number} {feature.name}")
    unit_labels.append(f"Unit {unit_number} on_creep")
for team in ["Blue", "Red"]:
    for upgrade in Upgrades:
        unit_labels.append(f"{team} {upgrade.name}")

## Get the SHAP Values

In [ ]:
shap_values = explainer.shap_values(shap_samples, nsamples=100)

## Create a SHAP Force Plot
NOTE: Attempting to create a sequential `force_plot()` may crash the notebook

In [ ]:
shap.force_plot(
    explainer.expected_value,
    shap_values[0, :],
    pd.Series(shap_samples[0], unit_labels)
)

## Create a SHAP Summary Plot

In [ ]:
shap.summary_plot(
    shap_values,
    pd.DataFrame(shap_samples, columns=unit_labels)
)